In [2]:
library(biomaRt)
library(data.table)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, first, last

The following object is masked from ‘package:biomaRt’:

    select

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



## Parsing the genome-wise gtf and conservation score tables

Preprocess the annotation file, isolate the 

* extract and classify information from *Homo_sapiens.GRCh38.92.gtf*
    - Filter the 'type' column to get gene/transcript
    - R/Py can both choose particular cols when parsing .csv(select = *columnNames*), which is a speed bonus for parsing

In [3]:
gtf_df  <- rtracklayer::import('Homo_sapiens.GRCh38.92.gtf')
gtf_df  <- as.data.frame(gtf_df)
genes = gtf_df[gtf_df$type == 'gene',]

In [48]:
# Save annotation for genes
write.table(genes, file = 'genes.GRCh38.92.gtf', sep = '\t')

In [70]:
# transcripts annotations
write.table(gtf_df[gtf_df$type == 'transcript',], file = 'transcript.GRCh38.92.gtf', sep = '\t')

In [3]:
colnames(gtf_df)

[1] "seqnames"                 "start"                   
 [3] "end"                      "width"                   
 [5] "strand"                   "source"                  
 [7] "type"                     "score"                   
 [9] "phase"                    "gene_id"                 
[11] "gene_version"             "gene_name"               
[13] "gene_source"              "gene_biotype"            
[15] "transcript_id"            "transcript_version"      
[17] "transcript_name"          "transcript_source"       
[19] "transcript_biotype"       "tag"                     
[21] "transcript_support_level" "exon_number"             
[23] "exon_id"                  "exon_version"            
[25] "protein_id"               "protein_version"         
[27] "ccds_id"

In [4]:
gene_colsOI = c('seqnames', 'start', 'end', 'width', 'strand', 'gene_id', 'gene_name', 'gene_biotype') 
# select columns of interest
genes = fread('genes.GRCh38.92.gtf', sep = '\t', select = gene_colsOI)

Warning message in fread("genes.GRCh38.92.gtf", sep = "\t", select = gene_colsOI):
“Detected 27 column names but the data has 28 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file.”

In [5]:
head(genes)

seqnames,start,end,width,strand,gene_id,gene_name,gene_biotype
1,11869,14409,2541,+,ENSG00000223972,DDX11L1,transcribed_unprocessed_pseudogene
1,14404,29570,15167,-,ENSG00000227232,WASH7P,unprocessed_pseudogene
1,17369,17436,68,-,ENSG00000278267,MIR6859-1,miRNA
1,29554,31109,1556,+,ENSG00000243485,MIR1302-2HG,lincRNA
1,30366,30503,138,+,ENSG00000284332,MIR1302-2,miRNA
1,34554,36081,1528,-,ENSG00000237613,FAM138A,lincRNA


In [6]:
trans_colsOI = c('seqnames', 'start', 'end', 'width', 'strand', 'gene_id', 'gene_name', 'gene_biotype', 'transcript_id', 'transcript_name', 'transcript_biotype')
transcripts = fread('transcript.GRCh38.92.gtf', sep = '\t', select = trans_colsOI) %>% as.data.frame()

Warning message in fread("transcript.GRCh38.92.gtf", sep = "\t", select = trans_colsOI):
“Detected 27 column names but the data has 28 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file.”

In [7]:
head(transcripts[transcripts$seqnames=='21',])

,seqnames,start,end,width,strand,gene_id,gene_name,gene_biotype,transcript_id,transcript_name,transcript_biotype
201193,21,5011799,5017145,5347,+,ENSG00000279493,FP565260.4,protein_coding,ENST00000624081,FP565260.4-201,protein_coding
201194,21,5022493,5040661,18169,+,ENSG00000277117,FP565260.3,protein_coding,ENST00000612610,FP565260.3-201,protein_coding
201195,21,5022493,5040666,18174,+,ENSG00000277117,FP565260.3,protein_coding,ENST00000620481,FP565260.3-205,protein_coding
201196,21,5022494,5036771,14278,+,ENSG00000277117,FP565260.3,protein_coding,ENST00000623960,FP565260.3-202,protein_coding
201197,21,5022531,5034684,12154,+,ENSG00000277117,FP565260.3,protein_coding,ENST00000623903,FP565260.3-203,nonsense_mediated_decay
201198,21,5022533,5036125,13593,+,ENSG00000277117,FP565260.3,protein_coding,ENST00000623795,FP565260.3-204,protein_coding


### Function 1 
* input: 
 - wig file path
 - chromosome 
 - genomic position ( nucleotide-wise)
 - distance d
* output:
 - A numeric parameter of length d, containing conservation scores upstream TSS.

In [8]:
# Returns the conservation score of a single TSS, within distance *d*. 
# Dependency: data.table(fread)
# TODO: write an alternate parameter that accepts gene id or gene name.
get_conservation_score <- function(wigpath, chr, pos, d){
    path = paste(wigpath, 'chr', as.character(chr), '.phyloP30way.wigFix', sep='')
    wigdata = fread(path)
    # use some regex to parse the first line of wig
    info = colnames(wigdata)
    chr_start = as.numeric(   substr(info, regexpr('start=', info)+6, regexpr(' step=', info))   )

#     tryCatch({
#             chr_start = as.numeric(   substr(info, regexpr('start=', info)+6, regexpr(' step=', info))   )
#         },error=function(e){cat("ERROR : Invalid wig file",conditionMessage(e),"\n")})
    
    # Transcription Start Site in .wig table
    TSSrow = pos - chr_start + 1    
    if( TSSrow-d<0 | TSSrow>nrow(wigdata)){
        print('Error: parameter out of range: d')
        return(NULL)
    }
    vec = as.numeric( wigdata[[1]][ (TSSrow-d) : TSSrow ])
    return(vec)
}

## Goal

 For a given chromosome *chr* and genomic distance d, this function maps conservation scores to all regions within d bp from the TSS of every gene in chromosome *chr*.
 * parse the conservation dataset to extract the relevant values according to genomic loci
 * do some validationt to verify that the anticipated output matches the actual output


In [9]:
# NO. of transcripts for each chromosome
# chr1 the longest; chr21 can be used for testing
for(i in c(1:22)){cat(paste(   i, ':', nrow(transcripts[transcripts$seqnames==as.character(i),])), '\n'   )}

1 : 18062 
2 : 14754 
3 : 12391 
4 : 8103 
5 : 9417 
6 : 8917 
7 : 9722 
8 : 7919 
9 : 6823 
10 : 6707 
11 : 12642 
12 : 11643 
13 : 3365 
14 : 7574 
15 : 7506 
16 : 10137 
17 : 12760 
18 : 3729 
19 : 12889 
20 : 4471 
21 : 2446 
22 : 4578 


In order to get a matrix at one time, revise the previous function:
* pass wig dataframes directly, instead of parsing .wig every time it's called
* append the output vector to a dataframe

## Function: *chr_phyloP(wigpath, chr, d, transcripts)*
### Arguments
* wigpath: Path to .wig tables(e.g. chr1.phyloP30way.wigFix, [UCSC download](http://hgdownload.cse.ucsc.edu/goldenpath/hg38/phyloP30way/hg38.30way.phyloP/ )). The path can be relative.
* chr: Chromosome number or 'X'/'Y'
* d: Desired distance from transcription start site

### Output
 A dataframe(matrix) with nrow==transcript-on-a-chr, ncol=d. Also contains gene id, transcript id, TSS, and transcript end site.


In [10]:

get_conservation_score <- function(pos, wigdata, chr_start, d, dat){
    # Transcription Start Site in .wig table
    TSSrow = pos - chr_start + 1 
    #   print(TSSrow)
    if( TSSrow-d<0 | TSSrow>nrow(wigdata)){
        print(paste('Error: parameter out of range: d, position =', pos))
        return(-1)
    }
    vec = rev(  as.numeric( wigdata[[1]][ (TSSrow-d) : TSSrow ])  )
    # rev(): let the cons-score vector in ascending order(0...d)
    dat = rbind( dat, vec )
    return(vec)
}

chr_phyloP <- function(wigpath, chr, d, transcripts){
    path = paste(  wigpath, 'chr', as.character(chr), '.phyloP30way.wigFix', sep='')
    wigdata = fread(path)
    info = colnames(wigdata)
    chr_start = as.numeric(   substr(info, regexpr('start=', info)+6, regexpr(' step=', info))    )
    # iterate all transcripts
    temp = transcripts[transcripts$start & transcripts$seqnames==as.character(chr),]
    TSS = temp$start # TSS vector on a chromosome
    
    cnames = c('chr', 'gene_id', 'transcript_id', 'start', 'end')
    dat = data.frame(matrix(ncol = d+1, nrow = 0))
    dat = sapply(TSS, get_conservation_score, wigdata=wigdata, chr_start=chr_start, d=d, dat=dat)
    dat = t(dat) %>% as.data.frame()
    
    colnames(dat) = as.character(c(0:d))
    
    chr = rep(as.character(chr), nrow(dat))
    gene_id = temp$gene_id
    transcript_id = temp$transcript_id
    start = temp$start
    end = temp$end
    dat = cbind(chr, gene_id, transcript_id, start, end, dat)
    #dat = cbind(chr, dat)
    return(dat)
}

In [18]:
Sys.time()
temp = chr_phyloP('wig/', 21, 4, transcripts)
Sys.time()

[1] "2018-08-19 04:04:20 CST"

[1] "2018-08-19 04:04:25 CST"

In [111]:
sessionInfo()

R version 3.4.3 (2017-11-30)
Platform: x86_64-redhat-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /usr/lib64/R/lib/libRblas.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] dplyr_0.7.6       data.table_1.11.4 biomaRt_2.34.2   

loaded via a namespace (and not attached):
 [1] Rcpp_0.12.16         pillar_1.2.1         bindr_0.1.1         
 [4] compiler_3.4.3       prettyunits_1.0.2    bitops_1.0-6        
 [7] base64enc_0.1-3      tools_3.4.3          progress_1.2.0      
[10] digest_0.6.15        uui

In [46]:
head(   transcripts[transcripts$seqnames=='21',])
transcripts[transcripts$seqnames=='21',][1000,]

,seqnames,start,end,width,strand,gene_id,gene_name,gene_biotype,transcript_id,transcript_name,transcript_biotype
201193,21,5011799,5017145,5347,+,ENSG00000279493,FP565260.4,protein_coding,ENST00000624081,FP565260.4-201,protein_coding
201194,21,5022493,5040661,18169,+,ENSG00000277117,FP565260.3,protein_coding,ENST00000612610,FP565260.3-201,protein_coding
201195,21,5022493,5040666,18174,+,ENSG00000277117,FP565260.3,protein_coding,ENST00000620481,FP565260.3-205,protein_coding
201196,21,5022494,5036771,14278,+,ENSG00000277117,FP565260.3,protein_coding,ENST00000623960,FP565260.3-202,protein_coding
201197,21,5022531,5034684,12154,+,ENSG00000277117,FP565260.3,protein_coding,ENST00000623903,FP565260.3-203,nonsense_mediated_decay
201198,21,5022533,5036125,13593,+,ENSG00000277117,FP565260.3,protein_coding,ENST00000623795,FP565260.3-204,protein_coding


,seqnames,start,end,width,strand,gene_id,gene_name,gene_biotype,transcript_id,transcript_name,transcript_biotype
202192,21,33465018,33467135,2118,-,ENSG00000142188,TMEM50B,protein_coding,ENST00000474272,TMEM50B-208,retained_intron


In [12]:
a =  fread('wig/chr21.phyloP30way.wigFix')
nrow(a)
head(a)

[1] 37275165

fixedStep chrom=chr21 start=5010001 step=1
0.338
0.369
0.326
0.300
0.300
0.369


In [17]:
a[86270:86290,]

fixedStep chrom=chr21 start=5010001 step=1
0.168
0.168
0.151
0.151
0.151
0.151
0.163
0.151
0.183
-1.688


### 致命问题：
* 读wig文件时不能整个作为dataframe读入（ phyloP46way 的 chr1 大小达到200G)。
* wig文件格式，有多个注释行，形如 'Fixedstep, start = chr = step=....' 需要改变方式来读取

### 综合以上两个问题，解决方案应当满足：
* 能认出注释
* 能处理注释中的信息
* 虽然不能一次性全部读入，但也可以有一定的缓冲区域，每次读一坨进来：
    - 对每一行对应 nucleotide (genomic coordinate) 有识别
    - 读取的 floating window 的数量级应该和 d 差不多甚至要大(?)
    
综上，我的思路(Augest 15th, 1pm)是：
* 读取文件时使用“指针”，每次向后移动到下一个TSS（偏移量）读取长度为d（实际操作中比d大很多）的score vector并记录。

第一遍处理的方法（找出header并识别信息）有官方C语言版本了：https://github.com/Ensembl/WiggleTools/blob/master/src/wigReader.c
methods:
* WiggleReaderReadHeader



In [28]:
test = fread('transcript.GRCh38.92.gtf', sep='\t')

Warning message in fread("transcript.GRCh38.92.gtf", sep = "\t"):
“Detected 27 column names but the data has 28 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file.”

In [29]:
unique(test$seqnames)

[1] "1"          "2"          "3"          "4"          "5"         
 [6] "6"          "7"          "X"          "8"          "9"         
[11] "11"         "10"         "12"         "13"         "14"        
[16] "15"         "16"         "17"         "18"         "20"        
[21] "19"         "Y"          "22"         "21"         "MT"        
[26] "KI270728.1" "KI270727.1" "KI270442.1" "GL000225.1" "GL000009.2"
[31] "GL000194.1" "GL000205.2" "GL000195.1" "KI270733.1" "GL000219.1"
[36] "GL000216.2" "KI270744.1" "KI270734.1" "GL000213.1" "GL000220.1"
[41] "GL000218.1" "KI270731.1" "KI270750.1" "KI270721.1" "KI270726.1"
[46] "KI270711.1" "KI270713.1"

In [30]:
temp

chr,gene_id,transcript_id,start,end,0,1,2,3,4
21,ENSG00000279493,ENST00000624081,5011799,5017145,0.744,0.744,0.823,0.653,-0.117
21,ENSG00000277117,ENST00000612610,5022493,5040661,0.684,-0.184,0.579,-0.319,0.512
21,ENSG00000277117,ENST00000620481,5022493,5040666,0.684,-0.184,0.579,-0.319,0.512
21,ENSG00000277117,ENST00000623960,5022494,5036771,-0.216,0.684,-0.184,0.579,-0.319
21,ENSG00000277117,ENST00000623903,5022531,5034684,-0.126,0.602,0.662,0.602,0.684
21,ENSG00000277117,ENST00000623795,5022533,5036125,0.756,-0.110,-0.126,0.602,0.662
21,ENSG00000279687,ENST00000623188,5073458,5087867,0.592,-1.049,-0.466,0.575,-0.808
21,ENSG00000280071,ENST00000624810,5079294,5127927,0.353,-1.449,-0.645,0.353,0.353
21,ENSG00000280071,ENST00000625036,5079294,5127927,0.353,-1.449,-0.645,0.353,0.353
21,ENSG00000280071,ENST00000620015,5116341,5127143,-0.125,-0.300,-1.018,-1.284,-0.687
